In [6]:
from google.cloud import bigquery
import pandas as pd # Importar pandas para uso explícito
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_absolute_percentage_error
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import plot_tree
import shap
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold


In [ ]:
from google.cloud import bigquery
import pandas as pd # Importar pandas para uso explícito



project_id = "modelagem-1971"
dataset_id = "ENEM"
table_name = "ENEM_2021_ONEHOT_V3"
table_path = f"{project_id}.{dataset_id}.{table_name}"
location = "us"

client = bigquery.Client(project=project_id, location=location)

sql_query = f"""
    SELECT * FROM `{table_path}` """ # Aumentamos o limite para ter mais dados no filtro

dataset_enem = client.query(sql_query).to_dataframe()

In [ ]:
dataset_enem.columns

Index(['TP_LINGUA', 'NU_NOTA_REDACAO', 'moradores_residencia',
       'TP_FAIXA_ETARIA_1', 'TP_FAIXA_ETARIA_2', 'TP_FAIXA_ETARIA_3',
       'TP_FAIXA_ETARIA_4', 'TP_FAIXA_ETARIA_5', 'TP_FAIXA_ETARIA_6',
       'TP_FAIXA_ETARIA_7',
       ...
       'tem_celular_E', 'tem_telefone_fixo_A', 'tem_telefone_fixo_B',
       'tem_computador_A', 'tem_computador_B', 'tem_computador_C',
       'tem_computador_D', 'tem_computador_E', 'tem_internet_A',
       'tem_internet_B'],
      dtype='object', length=210)

In [ ]:
dataset_enem_features = dataset_enem.drop(columns=['media_final'])
dataset_enem_target = dataset_enem['media_final']

X_train, X_test, y_train, y_test = train_test_split(
    dataset_enem_features, dataset_enem_target, test_size=0.20, random_state=42
)


KeyError: "['media_final'] not found in axis"

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

splits = 50

# Parâmetros de busca para XGBoost
param_grid_xgb = {
    'max_depth': np.arange(1, 21, step = 2), # Profundidade da árvore
    'learning_rate': np.arange(0.01, 0.08 ,step = 0.02),  # Taxa de aprendizado
    'n_estimators': np.arange(20, 100, step = 10),  # Número de árvores
    'subsample': np.arange(0.1, 0.8, step = 0.2),      # Fração de amostras usadas para cada árvore
    'colsample_bytree': np.arange(0.1, 0.7, step = 0.2),    # Fração de colunas usadas por árvore
    'gamma': np.arange(0.1, 0.6, step = 0.2),          # Regularização (evita overfitting)
    'reg_lambda': np.arange(0.1, 1, step = 0.2),       # Regularização L2
    'min_child_weight': np.arange(1, 10, step = 2)    # Peso mínimo para divisão de nós
}

# Validação cruzada
kf = KFold(n_splits=splits, shuffle=True, random_state=42)

# Scorer
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)
xgb_model = xgb.XGBRegressor(random_state=42)
random_search_xgb = RandomizedSearchCV(xgb_model, param_distributions=param_grid_xgb, cv=kf, scoring=scorer, n_jobs=-1, n_iter=20, random_state=42)

random_search_xgb.fit(X_train, y_train)

# Melhor modelo encontrado
melhor_modelo_xgb = random_search_xgb.best_estimator_

# Previsão
y_pred = melhor_modelo_xgb.predict(X_test)

# Avaliação
r2_xgb = r2_score(y_test, y_pred)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred))
mae_xgb = mean_absolute_error(y_test, y_pred)
mape_xgb = mean_absolute_percentage_error(y_test, y_pred)

metricas_resultados = pd.DataFrame({
    'Métrica': ['R²', 'RMSE', 'MAE', 'MAPE'],
    'Valor': [r2_xgb, rmse_xgb, mae_xgb, mape_xgb]
})

metricas_resultados['Valor'] = metricas_resultados['Valor'].apply(lambda x: f"{x:.4f}" if abs(x) < 1 else f"{x:,.4f}")
metricas_resultados.loc[metricas_resultados['Métrica'] == 'MAPE', 'Valor'] = metricas_resultados.loc[metricas_resultados['Métrica'] == 'MAPE', 'Valor'] + " %"

# Converter para Markdown
tabela_markdown = metricas_resultados.to_markdown(index=False)

# Salvar em arquivo .md
with open("metricas_resultados.md", "w", encoding="utf-8") as f:
    f.write(tabela_markdown)

# Resultados
print("Melhores parâmetros encontrados:")
print(random_search_xgb.best_params_)

ValueError: could not convert string to float: 'H'